# Book Club

## LIBRARIES AND SETTINGS

In [1]:
import pandas as pd
import requests
import re
from datetime import datetime
from bs4 import BeautifulSoup
import sqlite3

In [2]:
# simulates a browser
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# home page
url = 'https://books.toscrape.com/index.html'

## 1. DATA LOADING

### 1.1. Extraction the number of pages

In [3]:
# API request
page = requests.get(url, headers = headers)

# transform the html request into a beautiful soup object
soup = BeautifulSoup(page.text, 'html.parser')

# select the total number of pages
number_page = int(soup.find('li', class_ = 'current').text.split('of')[-1].strip())

### 1.2. Extraction of details page links for all books on the site

In [4]:
# creates the dataframe structure
df_books = pd.DataFrame(columns = ['link'])

for i in range(1, number_page + 1):
    # pagination
    url = 'https://books.toscrape.com/catalogue/page-' + str(i) + '.html'
    
    # API request
    page = requests.get(url, headers = headers)

    # transform the html request into a beautiful soup object
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # identifies the book
    book = soup.find_all('li', class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3')
    
    # extract the details url
    path = 'https://books.toscrape.com/catalogue/'
    book = [path + i.find('a')['href'] for i in book]
    
    # auxiliary dataframe 
    df_aux = pd.DataFrame(columns = ['link'])
    df_aux['link'] = book
    
    # contacts collected informations in a single dataframe
    df_books = pd.concat([df_books, df_aux], ignore_index = True)

### 1.3. Extract information from each book

In [5]:
for i in df_books.loc[:, 'link']:
    # API request
    page = requests.get(i, headers = headers)

    # transform the html request into a beautiful soup object
    soup = BeautifulSoup(page.text, 'html.parser')
        
    # informations
    category = soup.find('ul', class_ = 'breadcrumb').find_all('a')[2].get_text()
    title = soup.find('div', class_ = 'col-sm-6 product_main').find('h1').get_text()
    price = soup.find('div', class_ = 'col-sm-6 product_main').find('p', class_ = 'price_color').get_text()
    availability = soup.find('div', class_ = 'col-sm-6 product_main').find('p', class_ = 'instock availability').get_text().strip()
    rating = soup.find('div', class_ = 'col-sm-6 product_main').find_all('p')[2]['class'][-1]
    ucp = soup.find('table', class_ = 'table table-striped').find('td').get_text()

    # index
    line = df_books[df_books['link'] == i].index
    
    # put the data in the dataframe
    df_books.loc[line, 'category'] = category
    df_books.loc[line, 'title'] = title
    df_books.loc[line, 'price'] = price
    df_books.loc[line, 'availability'] = availability
    df_books.loc[line, 'rating'] = rating
    df_books.loc[line, 'ucp'] = ucp
    df_books['web_scraping_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

## 2. DATA CLEANING 

In [6]:
# --- PRICE 
# deletes strange character
df_books['price'] = df_books['price'].apply(lambda x: re.search('\d+\.\d+', x ).group(0) if pd.notnull(x) else x)

# converts the type of the variable
df_books['price'] = df_books['price'].astype(float)
    
# --- AVAILABILITY
# extracts the number
df_books['availability'] = df_books['availability'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

# converts the type of the variable
df_books['availability'] = df_books['availability'].astype(int)
    
# --- RATING
# changes word to number
df_books['rating'] = df_books['rating'].apply(lambda x: 1 if x == 'One' else
                                                        2 if x == 'Two' else
                                                        3 if x == 'Three' else
                                                        4 if x == 'Four' else 5)

# converts the type of the variable
df_books['rating'] = df_books['rating'].astype(int)
    
# --- DATE
# converts the type of the variable
df_books['web_scraping_date'] = pd.to_datetime(df_books['web_scraping_date'])
    
# --- LINK
# drops the feature link
df_books = df_books.drop('link', axis = 1)

## 3. DATABASE

### 3.1. Create table

In [8]:
query_bookclub_schema = """
                            CREATE TABLE IF NOT EXISTS books(
                                   ucp               TEXT,             
                                   category          TEXT,
                                   title             TEXT,
                                   price             REAL,
                                   rating            INTEGER,
                                   availability      INTEGER,
                                   web_scraping_date TEXT
                        );
                        """

# create table
conn = sqlite3.connect('../data/database_bookclub.sqlite')
cursor = conn.execute(query_bookclub_schema)
conn.commit()

### 3.2. Insert data

In [10]:
# organize the table
data_insert = df_books[['ucp', 'category', 'title', 'price', 'rating', 'availability', 'web_scraping_date']].copy()
    
# create database connection
conn = sqlite3.connect('../data/database_bookclub.sqlite')

# insert data
data_insert.to_sql('books', con = conn, if_exists = 'append', index = False)